In [36]:
# Imports

import numpy as np
import pandas as pd
from shared.utils import load_data
from datasets import preprocess_dataset, datasets_types

from sklearn.cluster import KMeans
from scipy.stats import beta as beta_dist
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from art.attacks.evasion import HopSkipJump

from art.estimators.classification.scikitlearn import ScikitlearnClassifier
from art.utils import to_categorical
from art.attacks.evasion import ZooAttack
from art.estimators.classification import SklearnClassifier
seed = 42

In [3]:
df = load_data(
            [
                "./shared/data/CIC_2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
                "./shared/data/CIC_2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
                "./shared/data/CIC_2017/Tuesday-WorkingHours.pcap_ISCX.csv"
            ]
        )
print("Dataset cargado")
df_preprocessed = preprocess_dataset(
    df, save=True, dataset_type="CIC_2017", seed=seed, load=False)
print("Dataset Preprocesado")

Dataset cargado
Loading new data
labels: {'-', 'FTP-Patator', 'SSH-Patator', 'Web Attack � XSS', 'Web Attack � Brute Force'}
Dataset Preprocesado


In [26]:
x_train = df_preprocessed.x_train
y_train = df_preprocessed.y_train
x_test = df_preprocessed.x_test
y_test = df_preprocessed.y_test

y_train = y_train.apply(lambda x: int(x))
y_test = y_test.apply(lambda x: int(x))

print(y_train.unique())
print(y_test.unique())

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

[0 1]
[0 1]
(633413, 63)
(633413,)
(271464, 63)
(271464,)


In [27]:
# Train a RF Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [28]:
# Train a MLP Classifier
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(32),max_iter=200,verbose=False,batch_size=200,early_stopping=True,activation='tanh',solver='adam')
mlp.fit(x_train, y_train)

MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=32)

In [29]:
# Train a NB Classifier
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)

GaussianNB()

In [30]:
# Train a DT Classifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [31]:
# Train a LR Classifier
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
FUNCTIONAL_FEATURES = [
 ' min_seg_size_forward',' Bwd Header Length',' Destination Port'
 'Init_Win_bytes_forward',' Init_Win_bytes_backward',' Bwd Packets/s'
 'Total Length of Fwd Packets',' Subflow Fwd Bytes',' Max Packet Length'
 'Bwd Packet Length Max',' Avg Bwd Segment Size',' Bwd Packet Length Mean'
 ' Fwd Packet Length Max',' Average Packet Size',' Packet Length Std'
 ' Packet Length Mean',' Bwd Packet Length Std',' Bwd Packet Length Min'
 ' Fwd Packet Length Std',' Fwd Packet Length Min',' Min Packet Length'
 ' Fwd Packet Length Mean',' Avg Fwd Segment Size',' act_data_pkt_fwd'
 ' Total Fwd Packets','Subflow Fwd Packets',' Total Backward Packets']
print(len(FUNCTIONAL_FEATURES))
FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c not in FUNCTIONAL_FEATURES][:-1]
print(FUNCTIONAL_FEATURES_IDEXES, len(FUNCTIONAL_FEATURES_IDEXES))
NON_FUNCTIONAL_FEATURES_IDEXES = [df.columns.get_loc(c) for c in df.columns if c in FUNCTIONAL_FEATURES]
print(NON_FUNCTIONAL_FEATURES_IDEXES, len(NON_FUNCTIONAL_FEATURES_IDEXES))
print(len(df.columns))

19
[0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 46, 49, 50, 51, 53, 55, 56, 57, 58, 59, 60, 61] 51
[3, 7, 13, 31, 43, 44, 45, 47, 48, 52, 54] 11
63


In [33]:
# Create ART classifier
class Classifier(ScikitlearnClassifier):
    
    def __init__(self, model, clip_values=None, preprocessing=(0, 1), attacks=[]):
        super(Classifier, self).__init__(model=model, clip_values=clip_values, preprocessing=preprocessing)
        self._attacks = attacks

    def predict(self, x, **kwargs):
        # Set attacks features to X
        for i in FUNCTIONAL_FEATURES_IDEXES:
            for j in range(len(x)):
                x[j][i] = self._attacks[j][i]
        predictions = self._model.predict(x)
        return to_categorical(predictions, nb_classes=self._get_nb_classes())

In [34]:
attacks_data = x_test[y_test == 1]
normal_data = x_test[y_test == 0]
print(attacks_data.shape, normal_data.shape)

(4815, 63) (266649, 63)


In [48]:


models = [rf, mlp, nb, dt, lr]
for model in models:
    # Create a SklearnClassifier wrapper around the trained classifier
    # classifier = Classifier(model=rf, clip_values=(0, 1), attacks=attacks_data)
    classifier = SklearnClassifier(model=model, clip_values=(0, 1))
    # Create the HopSkipJump attack
    attack = HopSkipJump(classifier=classifier, targeted=False,)
    # Generate adversarial data on the testing set
    x_test_adv = attack.generate(attacks_data[:100], np.zeros((attacks_data[:100].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:100], normal_data[:100]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:100].shape[0], 1)), np.zeros((normal_data[:100].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:100]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:100].shape[0], 1))))
    # Evaluate the Model with the adversarial data and the non adversarial data
    print("====================> Model: ", model)
    # print("---------- Non adversarial data")
    # print(classification_report(non_adv_y_test, model.predict(non_adv_x_test)))
    # print("Accuracy: ", accuracy_score(non_adv_y_test, model.predict(non_adv_x_test)))
    # print("Detection Rate: ", recall_score(non_adv_y_test, model.predict(non_adv_x_test)))
    # print("F1 Score: ", f1_score(non_adv_y_test, model.predict(non_adv_x_test)))
    # print("ROC AUC Score: ", roc_auc_score(non_adv_y_test, model.predict(non_adv_x_test)))
    print("---------- Adversarial data")
    print(classification_report(adv_y_test, model.predict(adv_x_test)))
    print("Accuracy: ", accuracy_score(adv_y_test, model.predict(adv_x_test)))
    print("Detection Rate: ", recall_score(adv_y_test, model.predict(adv_x_test)))
    print("F1 Score: ", f1_score(adv_y_test, model.predict(adv_x_test)))
    print("ROC AUC Score: ", roc_auc_score(adv_y_test, model.predict(adv_x_test)))



c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  RandomForestClassifier(random_state=42)
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67       100
         1.0       0.00      0.00      0.00       100

    accuracy                           0.50       200
   macro avg       0.25      0.50      0.33       200
weighted avg       0.25      0.50      0.33       200

Accuracy:  0.5
Detection Rate:  0.0
F1 Score:  0.0
ROC AUC Score:  0.5


c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

...

c:\Users\AntonioPayá\Desktop\Repos-Doctorado\apollon\venv\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  DecisionTreeClassifier()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67       100
         1.0       1.00      0.01      0.02       100

    accuracy                           0.51       200
   macro avg       0.75      0.51      0.34       200
weighted avg       0.75      0.51      0.34       200

Accuracy:  0.505
Detection Rate:  0.01
F1 Score:  0.019801980198019802
ROC AUC Score:  0.505


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  LogisticRegression()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.55      1.00      0.71       100
         1.0       1.00      0.18      0.31       100

    accuracy                           0.59       200
   macro avg       0.77      0.59      0.51       200
weighted avg       0.77      0.59      0.51       200

Accuracy:  0.59
Detection Rate:  0.18
F1 Score:  0.3050847457627119
ROC AUC Score:  0.59


# Apollon

In [44]:
class MultiArmedBanditThompsonSampling:

    def __init__(self, n_arms, n_clusters):
        self.n_arms = n_arms
        self.n_clusters = n_clusters
        self.arms = [RandomForestClassifier(), DecisionTreeClassifier(), GaussianNB()]
        self.cluster_centers = None
        self.cluster_assignments = None
        self.reward_sums = {}
        for cluster in range(n_clusters):
            self.reward_sums[cluster] = np.zeros(n_arms)
        self.alpha = np.ones(self.n_arms)
        self.beta = np.ones(self.n_arms)

    def train(self, X_train, y_train):
        kmeans = KMeans(n_clusters=self.n_clusters)
        self.cluster_assignments = kmeans.fit_predict(X_train)
        self.cluster_centers = kmeans.cluster_centers_
        # Print the number of samples in each cluster

        for i in range(self.n_clusters):
            print('Cluster {}: {}'.format(
                i, np.sum(self.cluster_assignments == i)))
            cluster_mask = self.cluster_assignments == i
            cluster_X_train = X_train[cluster_mask]
            cluster_y_train = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Training arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_train == arm
                arm_X_train = cluster_X_train[arm_mask]
                arm_y_train = cluster_y_train[arm_mask]
                if len(arm_X_train) > 0 and len(np.unique(arm_y_train)) > 1:
                    self.arms[arm].fit(arm_X_train, arm_y_train)
                else:
                    self.arms[arm].fit(X_train, y_train)

        # Set the arms rewards for each cluster
        for i in range(self.n_clusters):
            cluster_mask = self.cluster_assignments == i
            cluster_X_test = X_train[cluster_mask]
            cluster_y_test = y_train[cluster_mask]
            for arm in range(self.n_arms):
                print('Setting reward_sums arm {} on cluster {}'.format(arm, i))
                arm_mask = cluster_y_test == arm
                arm_X_test = cluster_X_test[arm_mask]
                arm_y_test = cluster_y_test[arm_mask]
                if len(arm_X_test) > 0:
                    arm_y_pred = self.arms[arm].predict(arm_X_test)
                    self.reward_sums[i][arm] = np.mean(
                        arm_y_pred == arm_y_test)

    def select_arm(self, cluster):
        # Select the arm with the highest reward
        theta = np.zeros(self.n_arms)
        for arm in range(self.n_arms):
            theta[arm] = np.random.beta(self.alpha[arm] + self.reward_sums[cluster]
                                        [arm], self.beta[arm] + 1 - self.reward_sums[cluster][arm])
        return np.argmax(theta)

    def predict(self, X_test):
        # Select the arm for each sample
        arms = np.zeros(len(X_test))
        for i in range(len(X_test)):
            cluster = np.argmin(np.linalg.norm(
                self.cluster_centers - X_test[i], axis=1))
            arms[i] = self.select_arm(cluster)
        # Predict using the selected arm
        y_pred = np.zeros(len(X_test))
        for arm in range(self.n_arms):
            arm_mask = arms == arm
            arm_X_test = X_test[arm_mask]
            if len(arm_X_test) > 0:
                y_pred[arm_mask] = self.arms[arm].predict(arm_X_test)
        return y_pred, arms


In [45]:
# Train the MAB
mab = MultiArmedBanditThompsonSampling(n_arms=3, n_clusters=2)
mab.train(x_train, y_train)

Cluster 0: 274888
Training arm 0 on cluster 0
Training arm 1 on cluster 0
Training arm 2 on cluster 0
Cluster 1: 358525
Training arm 0 on cluster 1
Training arm 1 on cluster 1
Training arm 2 on cluster 1
Setting reward_sums arm 0 on cluster 0
Setting reward_sums arm 1 on cluster 0
Setting reward_sums arm 2 on cluster 0
Setting reward_sums arm 0 on cluster 1
Setting reward_sums arm 1 on cluster 1
Setting reward_sums arm 2 on cluster 1


In [46]:
# Create a SklearnClassifier wrapper around the trained classifier
#classifier = Classifier(model=rf, clip_values=(0, 1), attacks=attacks_data)
models = [rf, mlp, nb, dt, lr]
accuracys = []
drs = []
f1s = []
rocs = []
for model in models:
    # Create a SklearnClassifier wrapper around the trained classifier
    # classifier = Classifier(model=rf, clip_values=(0, 1), attacks=attacks_data)
    classifier = SklearnClassifier(model=lr, clip_values=(0, 1))
    # Create the HopSkipJump attack
    attack = HopSkipJump(classifier=classifier, targeted=False,)
    # Generate adversarial data on the testing set
    x_test_adv = attack.generate(attacks_data[:100], np.zeros((attacks_data[:100].shape[0], 1)))

    non_adv_x_test = np.concatenate((attacks_data[:100], normal_data[:100]))
    non_adv_y_test = np.concatenate((np.ones((attacks_data[:100].shape[0], 1)), np.zeros((normal_data[:100].shape[0], 1))))
    adv_x_test = np.concatenate((x_test_adv, normal_data[:100]))
    adv_y_test = np.concatenate((np.ones((x_test_adv.shape[0], 1)), np.zeros((normal_data[:100].shape[0], 1))))


    # Evaluate the Model with the adversarial data and the non adversarial data
    print("====================> Model: ", model)
    # print("---------- Non adversarial data")
    # print(classification_report(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("Accuracy: ", accuracy_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # # Calculate the detection rate (attacks TP / attacks TP + attacks FN)
    # print("Detection Rate: ", recall_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("F1 Score: ", f1_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    # print("ROC AUC Score: ", roc_auc_score(non_adv_y_test, mab.predict(non_adv_x_test)[0]))
    print("---------- Adversarial data")
    accuracy = accuracy_score(adv_y_test, mab.predict(adv_x_test)[0])
    accuracys.append(accuracy)
    dr = recall_score(adv_y_test, mab.predict(adv_x_test)[0])
    drs.append(dr)
    f1 = f1_score(adv_y_test, mab.predict(adv_x_test)[0])
    f1s.append(f1)
    roc = roc_auc_score(adv_y_test, mab.predict(adv_x_test)[0])
    rocs.append(roc)
    print(classification_report(adv_y_test, mab.predict(adv_x_test)[0]))
    print("Accuracy: ", accuracy)
    print("Detection Rate: ", dr)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc)

print("=====================================")
print("Accuracy: ", sum(accuracys)/len(accuracys))
print("Detection Rate: ", sum(drs)/len(drs))
print("F1 Score: ", sum(f1s)/len(f1s))
print("ROC AUC Score: ", sum(rocs)/len(rocs))

HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  RandomForestClassifier(random_state=42)
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       100
         1.0       1.00      0.56      0.72       100

    accuracy                           0.78       200
   macro avg       0.85      0.78      0.77       200
weighted avg       0.85      0.78      0.77       200

Accuracy:  0.76
Detection Rate:  0.59
F1 Score:  0.7006369426751593
ROC AUC Score:  0.79


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  MLPClassifier(activation='tanh', batch_size=200, early_stopping=True,
              hidden_layer_sizes=32)
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.66      1.00      0.80       100
         1.0       1.00      0.49      0.66       100

    accuracy                           0.74       200
   macro avg       0.83      0.74      0.73       200
weighted avg       0.83      0.74      0.73       200

Accuracy:  0.75
Detection Rate:  0.52
F1 Score:  0.6533333333333333
ROC AUC Score:  0.775


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  GaussianNB()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.67      0.99      0.80       100
         1.0       0.98      0.51      0.67       100

    accuracy                           0.75       200
   macro avg       0.82      0.75      0.73       200
weighted avg       0.82      0.75      0.73       200

Accuracy:  0.77
Detection Rate:  0.46
F1 Score:  0.6486486486486487
ROC AUC Score:  0.75


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  DecisionTreeClassifier()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80       100
         1.0       1.00      0.50      0.67       100

    accuracy                           0.75       200
   macro avg       0.83      0.75      0.73       200
weighted avg       0.83      0.75      0.73       200

Accuracy:  0.725
Detection Rate:  0.53
F1 Score:  0.6164383561643836
ROC AUC Score:  0.77


HopSkipJump:   0%|          | 0/100 [00:00<?, ?it/s]

====================> Model:  LogisticRegression()
---------- Adversarial data
              precision    recall  f1-score   support

         0.0       0.66      0.99      0.79       100
         1.0       0.98      0.49      0.65       100

    accuracy                           0.74       200
   macro avg       0.82      0.74      0.72       200
weighted avg       0.82      0.74      0.72       200

Accuracy:  0.77
Detection Rate:  0.53
F1 Score:  0.6754966887417219
ROC AUC Score:  0.775
Accuracy:  0.7550000000000001
Detection Rate:  0.526
F1 Score:  0.6589107939126493
ROC AUC Score:  0.772
